In [6]:
import requests
import json

from dotenv import load_dotenv
import os
from shutil import copy
from pathlib import Path
from shutil import copy
import asyncio
import io
import aiohttp
from typing import List
from PIL import Image, ImageOps

import nest_asyncio
nest_asyncio.apply()

In [14]:

async def fetch_image(session, url):
    async with session.get(url) as response:
        img_bytes = await response.read()
        return img_bytes


async def download_images(image_urls: List[str],filenames,output_path) -> List[Image.Image]:
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image(session, url) for url in image_urls]
        images_bytes = await asyncio.gather(*tasks)
    
    print("Collected images' bytes!")
    images = []
    
    for image_bytes,filename in zip(images_bytes,filenames):
        
        img = ImageOps.exif_transpose(Image.open(io.BytesIO(image_bytes)).convert("RGB"))
        images.append(img)
        filename = filename.lower()
        filename = Path(filename + ".jpg")
        img.save(output_path/filename)
    
    return images

In [15]:
load_dotenv()

username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")


In [16]:
# store folders that have cereal boxes

dirs = ["/Users/iman/Downloads/ml-store-images/bf71326f-5168-48d9-9cd3-f7f5c32cd94c",
"/Users/iman/Downloads/ml-store-images/33eeaf0b-47da-4350-b8c6-ca499684da56",
"/Users/iman/Downloads/ml-store-images/59d021de-d665-443d-9ce5-5ba6d6a7e1c0",
"/Users/iman/Downloads/ml-store-images/160334d6-a310-4429-b460-c26e19de203e",
"/Users/iman/Downloads/ml-store-images/1830657b-f07a-4fee-8879-274113a9d7b7",
"/Users/iman/Downloads/ml-store-images/d910cc27-35a7-49e5-9c76-f0083f0e9345",
"/Users/iman/Downloads/ml-store-images/d428b039-9a8d-4e50-bae4-5a62596ba0bb",
"/Users/iman/Downloads/ml-store-images/b2b31caf-9e2b-4d73-bc5d-28890cb0d976",
"/Users/iman/Downloads/ml-store-images/1e0097c9-55f5-4667-8ad8-f00b5c715ea7",
"/Users/iman/Downloads/ml-store-images/7a31d692-8a0d-42b3-96f4-c81613ed0477",
"/Users/iman/Downloads/ml-store-images/eca17d09-9deb-45b6-88de-36b4f3fb83df",
"/Users/iman/Downloads/ml-store-images/57505c09-0472-4c75-90c5-e0e59986e088"]

stores = [store.split("/")[-1] for store in dirs]

In [17]:
training_dir = Path("/Users/iman/ml-datasets/detection_boxes/training_data")


In [ ]:
# training_dir = Path("/Users/iman/ml-datasets/detection_boxes/training_data")
# for directory in dirs:
#     directory = Path(directory)
#     for img in directory.glob('*.jpg'):
#         if "stitched_image" not in img.name:
#             copy(img,training_dir / img.name) 

In [7]:
login_url = "https://api.345.global/api/v1/login/access-token"
credentials = {"username": username, "password": password,"grant_type": "password"}
response = requests.post(login_url, data=credentials)
token_response = response.json()

In [86]:
training_images_paths = [path.name for path in (training_dir / "all_images").glob("*.jpg")]


In [ ]:
### query ml_store_images API endpoint and download all images

from PIL import Image
#access_token = token_response["access_token"]
token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)

# 2. Set headers
headers = {
    "Authorization": f"{token_type} {token}"
}

uuids_filter = ""
from urllib.parse import quote
annotations = []
images_api = set()
for store in stores:
    url = f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&store={store}"
    response = requests.get(url, headers=headers)
    response_data = response.json()
    store_images = response_data["data"]
    os.makedirs(training_dir / store,exist_ok=True)
    image_urls = [image["url"] for image in store_images if image["annotations"] != []]
    image_filenames = [image["uuid"] for image in store_images if image["annotations"] != []]
    asyncio.run(download_images(image_urls, image_filenames, training_dir / store))


Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!
Collected images' bytes!


In [13]:
good_uuids = [img.stem for img in Path("/Users/iman/ml-datasets/detection_boxes/drawn_bboxes/very good").glob("*.jpg")]
change_anno_uuids = [img.stem for img in Path("/Users/iman/ml-datasets/detection_boxes/drawn_bboxes/update_annotations").glob("*.jpg")]

In [ ]:
### create MLA files from images sorted into subdirs

#access_token = token_response["access_token"]
token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)

# 2. Set headers
headers = {
    "Authorization": f"{token_type} {token}"
}

uuids_filter = ""
from urllib.parse import quote
annotations = []
images_api = set()
for store in stores:
    url = f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&store={store}"
    response = requests.get(url, headers=headers)
    response_data = response.json()
    store_images = response_data["data"]
    for image in store_images:
        # if os.path.exists():
        #print(image["annotations"] == [])
        if image["annotations"] == []:
            continue
        if image["uuid"] in good_uuids:
            output_path = (Path("/Users/iman/ml-datasets/detection_boxes/mla_files/very_good") / image["uuid"]).with_suffix(".mla")    
            with open(output_path,"w") as f:
                json.dump(image,f)  
        elif image["uuid"] in change_anno_uuids:
            output_path = (Path("/Users/iman/ml-datasets/detection_boxes/mla_files/update_annotations") / image["uuid"]).with_suffix(".mla")    
            with open(output_path,"w") as f:
                json.dump(image,f)          


In [215]:
all_images = []
for store in training_dir.iterdir():
    if store.is_dir():
        for img in store.glob("*.jpg"):
            if img.name != ".DS_Store":
                all_images.append(img)

len(set(all_images))

684

In [ ]:
from torchvision import transforms
from torchvision.utils import draw_bounding_boxes, save_image
import torch
to_tensor = transforms.ToTensor()
for image_path in tqdm(training_dir.rglob("*.jpg")):
    image = to_tensor(Image.open(image_path).convert("RGB"))
    boxes = [annotation for annotation in annotations if annotation["uuid"] == image_path.stem][0]["bounding_boxes"]
    bboxes = []
    for i, box in enumerate(boxes):
        # if not box["face"]:
        #     continue
        bboxes.append([box["xmin"], box["ymin"], box["xmax"], box["ymax"]])
    bboxes = torch.tensor(bboxes, dtype=torch.float32).reshape(-1, 4)
    bboxes = bboxes * torch.tensor([image.shape[2], image.shape[1], image.shape[2], image.shape[1]], dtype=torch.float32)
    drawn = draw_bounding_boxes(image,bboxes,width=15,colors="red")
    save_image(drawn, training_dir.parent / "drawn_bboxes" / f"{image_path.stem}.jpg")

684it [01:24,  8.12it/s]


In [ ]:
to_tensor = transforms.ToTensor()
for image_path in tqdm(training_dir.rglob("*.jpg")):
    image = to_tensor(Image.open(image_path).convert("RGB"))
    boxes = [annotation for annotation in annotations if annotation["uuid"] == image_path.stem][0]["bounding_boxes"]
    bboxes = []
    for i, box in enumerate(boxes):
        if not box["face"]:
            continue
        bboxes.append([box["xmin"], box["ymin"], box["xmax"], box["ymax"]])
    bboxes = torch.tensor(bboxes, dtype=torch.float32).reshape(-1, 4)
    bboxes = bboxes * torch.tensor([image.shape[2], image.shape[1], image.shape[2], image.shape[1]], dtype=torch.float32)
    drawn = torchvision.utils.draw_bounding_boxes(image,bboxes,width=15,colors="red")
    torchvision.utils.save_image(drawn, training_dir.parent / "drawn_bboxes_no_faces" / f"{image_path.stem}.jpg")

In [ ]:
for image in annotations:
    output_path = (Path("/Users/iman/ml-datasets/detection_boxes/mla_files") / image["uuid"]).with_suffix(".mla")
    # with open(output_path,"w") as f:
    #     json.dump(image,f)

{'filename': 'IMG_2989.JPG', 'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/eb10488eda67d74a7bc74946fb64b2f9.jpg', 'uuid': 'c04e55b6-d524-4df8-967a-2e16f4259b65', 'width': 3024, 'height': 4032, 'bounding_boxes': [{'class': 'Object', 'xmin': 0.2161, 'xmax': 0.427, 'ymin': 0.8658, 'ymax': 0.9594, 'face': 8, 'orientation': 8}, {'class': 'Object', 'xmin': 0.2064, 'xmax': 0.433, 'ymin': 0.7596, 'ymax': 0.8637, 'face': 8, 'orientation': 8}, {'class': 'Object', 'xmin': 0.1899, 'xmax': 0.345, 'ymin': 0.0852, 'ymax': 0.1667, 'face': 1, 'orientation': 0}, {'class': 'Object', 'xmin': 0.8213, 'xmax': 0.979, 'ymin': 0.102, 'ymax': 0.1832, 'face': 1, 'orientation': 0}, {'class': 'Object', 'xmin': 0.6826, 'xmax': 0.974, 'ymin': 0.5827, 'ymax': 0.7076, 'face': 1, 'orientation': 0}, {'class': 'Object', 'xmin': 0.8546, 'xmax': 0.993, 'ymin': 0.2841, 'ymax': 0.3653, 'face': 1, 'orientation': 0}, {'class': 'Object', 'xmin': 0.1761, 'xmax': 0.378, 'ymin': 0.1969, 'ymax': 0.349, 'fac

In [ ]:
## re download images for validated/cleaned MLAs
for folder in Path("/Users/iman/Downloads/vq-mla_files-download/MLA files").iterdir():
    urls = []
    uuids = []
    output_path = Path("/Users/iman/ml-datasets/detection_boxes/mla_files/" + folder.name + "_checked")
    output_path.mkdir(parents=True, exist_ok=True)
    for file in folder.rglob("*.mla"):
        with open(file) as f:
            data = json.load(f)
        urls.append(data["url"])
        uuids.append(data["uuid"])
    asyncio.run(download_images(urls, uuids, output_path))


Collected images' bytes!
Collected images' bytes!
Collected images' bytes!


In [27]:
all_images_paths = []
for image_path in Path("/Users/iman/ml-datasets/detection_boxes/all_checked").rglob("*.jpg"):
    all_images_paths.append(image_path.stem)

In [34]:
training_uuids = [mla_path.stem for mla_path in Path("/Users/iman/ml-datasets/detection_boxes/mla_files/all_checked_filtered").rglob("*.mla")]

In [54]:
f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&filter={quote(uuids_filter)}"

'https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&filter=uuid%3D4abae850-77c7-46eb-b8d9-4cfc3d5ed34d%2Cuuid%3Dba0ce192-4004-4f6d-88ea-445c9f5228f9%2Cuuid%3D41a94560-89e2-4c71-8586-95a09100e6e4%2Cuuid%3D8ccedb7d-17c3-4d20-929e-879a53bf5797%2Cuuid%3Df630fb54-e7d9-4156-be03-ee3f6c572cf6%2Cuuid%3Db810bd29-8b90-437c-968c-eaa727a0f1be%2Cuuid%3D2302d787-5b63-4d47-beb3-2555a7743f93%2Cuuid%3D62672095-fb34-40e8-bf3d-1ea7be814285%2Cuuid%3D2e56d25a-351c-47c6-8456-f7ab5b59e1a8%2Cuuid%3Db3c18d4f-7fd2-4828-b3ca-4469a4e24ea5%2Cuuid%3Df80544dd-8d94-4165-b18a-d33417eb892b%2Cuuid%3D9d0f6c62-917d-4f43-90ce-d15891ef3dc7%2Cuuid%3Da41671f7-5311-4fbe-92fa-a80ce15bb9c8%2Cuuid%3De7d02430-2d04-499a-8d1e-429af64f8393%2Cuuid%3D9b055ef8-204e-4bb1-a689-3ff4c9bb8b08%2Cuuid%3D7c8574cc-9ed6-4153-be75-b7576dd89891%2Cuuid%3D2ac56eaf-0a4d-4672-9b3c-872cdeb55e03%2Cuuid%3D52d9b5a5-5f1f-4f2c-b412-5c5076064b3e%2Cuuid%3D1444f882-6bc7-446f-aecc-82e161611b73%2Cuuid%3D9873c102-024a-4037-96ea-42edc0d6

In [ ]:
# if using annotations directly from VQ
# don't use if you've cleaned annotations 

login_url = "https://api.345.global/api/v1/login/access-token"
credentials = {"username": username, "password": password,"grant_type": "password"}
response = requests.post(login_url, data=credentials)
token_response = response.json()

from PIL import Image
#access_token = token_response["access_token"]
token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)

# 2. Set headers
headers = {
    "Authorization": f"{token_type} {token}"
}

uuids_filter = ""
for uuid in training_uuids:
    uuids_filter += f"uuid={uuid},"
uuids_filter = uuids_filter[:-1]  # Remove the last comma

from urllib.parse import quote
annotations = []
images_api = set()
url = f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&filter={quote(uuids_filter)}"
response = requests.get(url, headers=headers)
response_data = response.json()
print(response_data)
all_images = response_data["data"]
print(len(all_images))

for image in all_images:

    filename = image["filename"]
    uuid = image["uuid"]
    # # Skip stitched images and duplicates
    # if "stitched_image" in filename:
    #     continue

    entry = {
        "filename": filename,
        "url": image["url"],
        "uuid": uuid,
        "width": None,
        "height": None,
        "bounding_boxes": [],
        "store": store
    }
    uuid_path = uuid + ".jpg"
    image_path = Path("/Users/iman/ml-datasets/detection_boxes/all_checked_filtered") / uuid_path
    if image_path.exists():
        w, h = Image.open(image_path).size
        entry["width"] = w
        entry["height"] = h

        for annotation in image.get("annotations", []):
            anno = {
                "class": "Object",
                "xmin": annotation["xmin"],
                "xmax": annotation["xmax"],
                "ymin": annotation["ymin"],
                "ymax": annotation["ymax"],
                "face": annotation["face"],
                "orientation": annotation["orientation"]
            }
            entry["bounding_boxes"].append(anno)

        annotations.append(entry)

{'total_count': 130, 'page_count': 130, 'query_count': 130, 'data': [{'uuid': '77a2782a-b91c-4440-958c-a874ff573391', 'fileHash': '01fd46632eedf62ffb2edc35a7bb61d9f60802b3806dbeb2488c69bec854d42c', 'fileHashMd5': 'c1447b2a8941d88a473f99a451f96f3e', 'filename': 'IMG_3429.JPG', 'extension': 'jpg', 'folder': 'ml_store_images', 'size': 1897880, 'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/c1447b2a8941d88a473f99a451f96f3e.jpg', 'scope': None, 'lat': 44.87403055555556, 'lng': -93.324325, 'upcLocations': None, 'surveyJobUuid': None, 'createdAt': '2020-10-28T01:59:43.911300+00:00', 'updatedAt': '2020-10-28T02:44:56.697302+00:00', 'updateIndex': -1290, 'deletedAt': None, 'createdBy': None, 'store': {'uuid': '160334d6-a310-4429-b460-c26e19de203e', 'name': 'Mark 1', 'retailerName': None, 'address': None, 'city': None, 'state': None, 'zipCode': None}, 'annotations': [{'uuid': 'caa8a5e7-cbb2-42bd-b62b-b0af96625cd7', 'xmin': 0.7238, 'xmax': 0.822, 'ymin': 0.0467, 'ymax': 0.

In [ ]:
from urllib.parse import quote

login_url = "https://api.345.global/api/v1/login/access-token"
credentials = {"username": username, "password": password,"grant_type": "password"}
response = requests.post(login_url, data=credentials)
token_response = response.json()

from PIL import Image
#access_token = token_response["access_token"]
token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)
# 2. Set headers
headers = {
    "Authorization": f"{token_type} {token}"
}

url = f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&filter={quote(f"uuid=4abae850-77c7-46eb-b8d9-4cfc3d5ed34d")}"
response = requests.get(url, headers=headers)
response_data = response.json()
print(response_data)
all_images = response_data["data"]


{'total_count': 1, 'page_count': 1, 'query_count': 1, 'data': [{'uuid': '4abae850-77c7-46eb-b8d9-4cfc3d5ed34d', 'fileHash': '7b7bd0c6e1dc9f726a37d63a5ca1de0aa7b57aae4958911e0700683373c1e9f1', 'fileHashMd5': '28b1ff4bf20fde6dab18d654ca7b38ea', 'filename': 'IMG_0080.JPG', 'extension': 'jpg', 'folder': 'ml_store_images', 'size': 510686, 'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/28b1ff4bf20fde6dab18d654ca7b38ea.jpg', 'scope': None, 'lat': 35.93865833333333, 'lng': -86.81939722222222, 'upcLocations': None, 'surveyJobUuid': None, 'createdAt': '2020-10-28T02:00:37.496189+00:00', 'updatedAt': '2020-10-28T18:52:26.388165+00:00', 'updateIndex': -1339, 'deletedAt': None, 'createdBy': None, 'store': {'uuid': '1830657b-f07a-4fee-8879-274113a9d7b7', 'name': 'Kendall 1', 'retailerName': None, 'address': None, 'city': None, 'state': None, 'zipCode': None}, 'annotations': [{'uuid': 'a5b0f093-ec9a-463e-afce-b20fdb17c976', 'xmin': 0.4772, 'xmax': 0.651, 'ymin': 0.5218, 'ymax'

[{'uuid': '4abae850-77c7-46eb-b8d9-4cfc3d5ed34d',
  'fileHash': '7b7bd0c6e1dc9f726a37d63a5ca1de0aa7b57aae4958911e0700683373c1e9f1',
  'fileHashMd5': '28b1ff4bf20fde6dab18d654ca7b38ea',
  'filename': 'IMG_0080.JPG',
  'extension': 'jpg',
  'folder': 'ml_store_images',
  'size': 510686,
  'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/28b1ff4bf20fde6dab18d654ca7b38ea.jpg',
  'scope': None,
  'lat': 35.93865833333333,
  'lng': -86.81939722222222,
  'upcLocations': None,
  'surveyJobUuid': None,
  'createdAt': '2020-10-28T02:00:37.496189+00:00',
  'updatedAt': '2020-10-28T18:52:26.388165+00:00',
  'updateIndex': -1339,
  'deletedAt': None,
  'createdBy': None,
  'store': {'uuid': '1830657b-f07a-4fee-8879-274113a9d7b7',
   'name': 'Kendall 1',
   'retailerName': None,
   'address': None,
   'city': None,
   'state': None,
   'zipCode': None},
  'annotations': [{'uuid': 'a5b0f093-ec9a-463e-afce-b20fdb17c976',
    'xmin': 0.4772,
    'xmax': 0.651,
    'ymin': 0.5218,


In [12]:
all_images[0]

{'uuid': '4abae850-77c7-46eb-b8d9-4cfc3d5ed34d',
 'fileHash': '7b7bd0c6e1dc9f726a37d63a5ca1de0aa7b57aae4958911e0700683373c1e9f1',
 'fileHashMd5': '28b1ff4bf20fde6dab18d654ca7b38ea',
 'filename': 'IMG_0080.JPG',
 'extension': 'jpg',
 'folder': 'ml_store_images',
 'size': 510686,
 'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/28b1ff4bf20fde6dab18d654ca7b38ea.jpg',
 'scope': None,
 'lat': 35.93865833333333,
 'lng': -86.81939722222222,
 'upcLocations': None,
 'surveyJobUuid': None,
 'createdAt': '2020-10-28T02:00:37.496189+00:00',
 'updatedAt': '2020-10-28T18:52:26.388165+00:00',
 'updateIndex': -1339,
 'deletedAt': None,
 'createdBy': None,
 'store': {'uuid': '1830657b-f07a-4fee-8879-274113a9d7b7',
  'name': 'Kendall 1',
  'retailerName': None,
  'address': None,
  'city': None,
  'state': None,
  'zipCode': None},
 'annotations': [{'uuid': 'a5b0f093-ec9a-463e-afce-b20fdb17c976',
   'xmin': 0.4772,
   'xmax': 0.651,
   'ymin': 0.5218,
   'ymax': 0.7894,
   'annota

In [20]:
# if using annotations from local cleaned 
annotations = []

for mla_path in Path("/Users/iman/ml-datasets/detection_boxes/misc/mla_files/all_checked_filtered").rglob("*.mla"):
    with open(mla_path) as f:
        image = json.load(f)
        print(mla_path,image)
        uuid = image["uuid"]
        # # Skip stitched images and duplicates
        # if "stitched_image" in filename:
        #     continue
        filename = image.get("filename")
        store = image.get("store")
        if not filename or not store:
            url = f"https://api.345.global/api/v1/productMLStoreImages/paginate?skip=0&limit=10000&filter={quote(f"uuid={uuid}")}"
            response = requests.get(url, headers=headers)
            response_data = response.json()
            image_data = response_data["data"]
            filename = image_data[0]["filename"]
            store = image_data[0]["store"]
        
        entry = {
            "filename": filename,
            "url": image["url"],
            "uuid": uuid,
            "width": None,
            "height": None,
            "bounding_boxes": [],
            "store": store
        }
        uuid_path = uuid + ".jpg"
        image_path = Path("/Users/iman/ml-datasets/detection_boxes/misc/all_checked_filtered") / uuid_path
        if image_path.exists():
            w, h = Image.open(image_path).size
            entry["width"] = w
            entry["height"] = h

            for annotation in image.get("annotations", []):
                anno = {
                    "class": "Object",
                    "xmin": annotation["xmin"],
                    "xmax": annotation["xmax"],
                    "ymin": annotation["ymin"],
                    "ymax": annotation["ymax"],
                    "face": annotation.get("face",0),
                    "orientation": annotation.get("orientation",0)
                }
                entry["bounding_boxes"].append(anno)

            annotations.append(entry)

/Users/iman/ml-datasets/detection_boxes/misc/mla_files/all_checked_filtered/4abae850-77c7-46eb-b8d9-4cfc3d5ed34d.mla {'version': 2, 'uuid': '4abae850-77c7-46eb-b8d9-4cfc3d5ed34d', 'url': 'https://storage.googleapis.com/core-345-assets/ml_store_images/28b1ff4bf20fde6dab18d654ca7b38ea.jpg', 'mlInfo': None, 'annotations': [{'uuid': '68da38aa-f614-4a7d-91fc-5f2c91594ad0', 'xmin': 0.2749, 'ymin': 0.5096, 'xmax': 0.477, 'ymax': 0.7871, 'predictions': [], 'selectedReference': None, 'annotationType': 'product'}, {'uuid': '231931fa-eb51-422d-940d-4ab4e9f6e94c', 'xmin': 0.0139, 'ymin': 0.1314, 'xmax': 0.156, 'ymax': 0.3715, 'predictions': [], 'selectedReference': None, 'annotationType': 'product'}, {'uuid': 'c9549629-b733-46e7-a18f-967241519241', 'xmin': 0.1609, 'ymin': 0.1352, 'xmax': 0.289, 'ymax': 0.372, 'predictions': [], 'selectedReference': None, 'annotationType': 'product'}, {'uuid': '505120e8-ecc2-40ea-a358-8303cc75e816', 'xmin': 0.7673, 'ymin': 0.1388, 'xmax': 0.892, 'ymax': 0.3747, 'pr

In [21]:
len(annotations)

130

In [22]:
training_dir

PosixPath('/Users/iman/ml-datasets/detection_boxes/training_data')

In [23]:
json.dump(annotations, open(Path("/Users/iman/ml-datasets/detection_boxes/vq_boxes/training_data") / "annotations_1.json", "w"), indent=4)
